## 01 - Data Cleaning

*Get a general overview of the data, clean if needed, and prepare for exploration*

In [1]:
from pathlib import Path
import pandas as pd
from IPython.display import display

In [ ]:
project_root = Path.cwd().parent
raw_data = project_root / 'data' / 'raw'

df = pd.read_csv(raw_data / 'walsoft_semi_categorized_phone_dataset.csv')
df_categories = pd.read_csv(raw_data / 'categories.csv')

In [3]:
display(df_categories)

,category,description,example_contacts
0,Unknown,"Mixed, uncategorised or infrequent contacts. M...","One-off callers, rare repeat numbers, mixed be..."
1,Family,Relatives and close family members. Typically ...,"Daughter, son, spouse, sibling, parent"
2,Supplier,"Business suppliers and vendors. Mostly short, ...","Stationery supplier, IT parts vendor, printing..."
3,Important Contacts,Key practical and operational contacts who are...,"Driver, tenant, caretaker, accountant, key helper"
4,Service Provider,Formal institutions and service organisations ...,"Bank, insurance company, SARS, Department of L..."


In [4]:
display(df)

,date_stamp,time,day-of_week,month,year,dialled_phone_number,name,duration_in_seconds,category
0,1/1/2022,16:03:01,Saturday,January,2022,648578192,Abel,179,Unknown
1,1/1/2022,16:06:17,Saturday,January,2022,814500001,Husband CEL01,66,Family
2,1/1/2022,19:08:44,Saturday,January,2022,814500001,Husband CEL01,38,Family
3,1/1/2022,20:03:11,Saturday,January,2022,694500003,Daughter MAS01,1283,Family
4,1/2/2022,14:22:44,Sunday,January,2022,814500001,Husband CEL01,66,Family
...,...,...,...,...,...,...,...,...,...
24947,10/4/2024,19:33:03,Friday,October,2024,680900016,Daughter ANG01,22,Family
24948,10/4/2024,20:06:28,Friday,October,2024,826902002,Accountant - Tom,12,Important Contacts
24949,10/4/2024,20:06:50,Friday,October,2024,826902002,Accountant - Tom,4,Important Contacts
24950,10/4/2024,20:07:53,Friday,October,2024,826902002,Accountant - Tom,2,Important Contacts


Eish! We see an annoying hyphen in the *day-of_week* column, lets fix it because i love standarized stuff

In [5]:
df = df.rename(columns={'day-of_week': 'day_of_week'})

In [6]:
df.describe()

,year,dialled_phone_number,duration_in_seconds
count,24952.000000,2.495200e+04,24952.000000
mean,2022.822940,7.021512e+08,103.140430
std,0.767296,1.653469e+08,287.153126
min,2022.000000,1.000003e+08,1.000000
25%,2022.000000,6.714000e+08,9.000000
50%,2023.000000,7.310754e+08,34.000000
75%,2023.000000,8.142004e+08,84.000000
max,2024.000000,8.782482e+08,7200.000000


- Majority of calls were made in 2022
- Longest call lasted 7200 seconds (2 hours), shortest 1 second, with a mean of 103 seconds

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24952 entries, 0 to 24951
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date_stamp            24952 non-null  object
 1   time                  24952 non-null  object
 2   day_of_week           24952 non-null  object
 3   month                 24952 non-null  object
 4   year                  24952 non-null  int64 
 5   dialled_phone_number  24952 non-null  int64 
 6   name                  24952 non-null  object
 7   duration_in_seconds   24952 non-null  int64 
 8   category              24952 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.7+ MB



We see column *date_stamp* are of datatype object, let's change *date_stamp* to be of datetime, and add an *hour* column of type integer

In [8]:
df['date_stamp'] = pd.to_datetime(df['date_stamp'])
df['hour'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.hour

Let's add a boolean column to flag whether a call was made during business hours. We'll define business hours as 8am to 17pm (5pm), Monday through Friday

In [9]:
df['is_business_hours'] = (
    (df['hour'] >= 8) &
    (df['hour'] < 17) &
    (df['day_of_week'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']))
)

In [10]:
missing = df.isnull().sum()
print(missing)

date_stamp              0
time                    0
day_of_week             0
month                   0
year                    0
dialled_phone_number    0
name                    0
duration_in_seconds     0
category                0
hour                    0
is_business_hours       0
dtype: int64


In [11]:
duplicates = df.duplicated().sum()
print(duplicates)

0


A responsible kaggle user has uploaded this dataset, no missing values or duplicates

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24952 entries, 0 to 24951
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date_stamp            24952 non-null  datetime64[ns]
 1   time                  24952 non-null  object        
 2   day_of_week           24952 non-null  object        
 3   month                 24952 non-null  object        
 4   year                  24952 non-null  int64         
 5   dialled_phone_number  24952 non-null  int64         
 6   name                  24952 non-null  object        
 7   duration_in_seconds   24952 non-null  int64         
 8   category              24952 non-null  object        
 9   hour                  24952 non-null  int32         
 10  is_business_hours     24952 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int32(1), int64(3), object(5)
memory usage: 1.8+ MB


Great, data is now clean and ready to be added to our /data/processed folder

In [ ]:
df.to_csv(project_root / 'data' / 'processed' / 'calls_cleaned.csv', index=False)